In [ ]:
import bs4
from langchain.chat_models import init_chat_model
from langchain_openai import OpenAIEmbeddings
from langchain_core.vectorstores import InMemoryVectorStore
from langchain import hub
from langchain_community.document_loaders import WebBaseLoader
from langchain_core.documents import Document
from langchain_text_splitters import RecursiveCharacterTextSplitter
from typing import List, TypedDict
from langgraph.graph import StateGraph, MessagesState, END
from langchain_core.tools import tool
from langchain_core.messages import SystemMessage
from langgraph.prebuilt import ToolNode, tools_condition
from langgraph.checkpoint.memory import MemorySaver

from dotenv import load_dotenv
from IPython.display import Image, display

In [2]:
load_dotenv()

True

In [60]:
llm = init_chat_model("gemini-2.5-flash", model_provider="google_genai")
embeddings = OpenAIEmbeddings(model="text-embedding-3-large")
# vector_store = InMemoryVectorStore(embeddings)

In [10]:
loader = WebBaseLoader(
    web_paths=("https://lilianweng.github.io/posts/2023-06-23-agent/",),
    bs_kwargs=dict(
        parse_only=bs4.SoupStrainer(
            class_=("post-content", "post-title", "post-header")
        )
    ),
)
docs = loader.load()
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
chunks = text_splitter.split_documents(docs)
print(f"From {len(docs)} docs created {len(chunks)} chunks.")

From 1 docs created 63 chunks.


In [14]:
vectorstore = InMemoryVectorStore.from_documents(chunks, embeddings)
print(f"Create VectorStore with {len(vectorstore.store.items())}")

Create VectorStore with 63


In [69]:
@tool(response_format="content_and_artifact")
def retrieve(query: str):
    """Retrieve information related to a query."""
    retrieved_docs = vectorstore.similarity_search(query, k=2)
    serialized = "\n\n".join((f"Source: {doc.metadata}\nContent: {doc.page_content}") for doc in retrieved_docs)
    return serialized, retrieved_docs



In [89]:
def query_or_respond(state: MessagesState):
    llm_with_tools = llm.bind_tools([retrieve])
    response = llm_with_tools.invoke(state["messages"])
    return {"messages": [response]}

tools = ToolNode([retrieve])

def generate(state: MessagesState):
    recent_tool_messages = []
    for message in reversed(state["messages"]):
        if message.type == "tool":
            recent_tool_messages.append(message)
        else:
            break
    tool_messages = recent_tool_messages[::-1]

    docs_content = "\n\n".join(doc.content for doc in tool_messages)
    system_message_content = (
        "You are an assistant for question-answering tasks. "
        "Use the following pieces of retrieved context to answer "
        "the question. If you don't know the answer, say that you "
        "don't know. Use three sentences maximum and keep the "
        "answer concise."
        "\n\n"
        f"{docs_content}"
    )

    conversation_messages = [
        message for message in state["messages"]
        if message.type in ("human", "system")
        or (message.type == "ai" and not message.tool_calls)
    ]
    prompt = [SystemMessage(system_message_content)] + conversation_messages

    response = llm.invoke(prompt)
    return {"messages": [response]}

In [90]:
graph_builder = StateGraph(MessagesState)
graph_builder.add_node(query_or_respond)
graph_builder.add_node(tools)
graph_builder.add_node(generate)
graph_builder.set_entry_point("query_or_respond")
graph_builder.add_conditional_edges(
    "query_or_respond",
    tools_condition,
    {END: END, "tools": "tools"},
)

graph_builder.add_edge("tools", "generate")
graph_builder.add_edge("generate", END)

memory = MemorySaver()
graph = graph_builder.compile(checkpointer=memory)
# display(Image(graph.get_graph().draw_mermaid_png()))

config = {"configurable": {"thread_id": "abc123"}}

In [ ]:
input = "What is Task Decomposition?"
resp = graph.invoke({"messages": [{'role': 'user', 'content': input}]}, config=config)
# print(resp['messages'][-1].pretty_print())
print(resp['messages'][-1].content)

input = "Could you look up some common way of doing it?"
resp = graph.invoke({"messages": [{'role': 'user', 'content': input}]}, config=config)
print(resp['messages'][-1].pretty_print())


In [ ]:
for step in graph.stream(
    {"messages": [{'role': 'user', 'content': input}]},
    stream_mode="values",
    config=config
):
    step["messages"][-1].pretty_print()

In [ ]:
input = "Could you look up some common way of doing it?"

for step in graph.stream(
    {"messages": [{'role': 'user', 'content': input}]},
    stream_mode="values",
    config=config
):
    step["messages"][-1].pretty_print()

In [96]:
import gradio as gr

def chat(message, history):
    resp = graph.invoke({"messages": [{"role": 'user', 'content': message}]}, config=config)
    return resp["messages"][-1].content

ui = gr.ChatInterface(fn=chat, type="messages")
ui.launch(inbrowser=False)

* Running on local URL:  http://127.0.0.1:7860
* To create a public link, set `share=True` in `launch()`.
